- **Project**: UAS Pembelajaran Mesin
- **Created**: Sun, 18th Dec 2022 1:15:24 PM
- **Author**: F. Waskito
- **Reference**: Randerson. (July 12, 2019). Classifiy Images Using Convolutional Neural Network & Python. Medium.com. https://randerson112358.medium.com/classify-images-using-convolutional-neural-networks-python-a89cecc8c679

Impor pustaka yang dibutuhkan

In [ ]:
# Impor pustaka yang dibutuhkan
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras import layers
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

plt.style.use('fivethirtyeight')

*Outset*

In [1]:
os.getcwd()

'/home/waskito/project/gen/pm/uas'

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

*Load data*

In [ ]:
from keras.datasets import cifar10
# Sumber data (cifar10) telah diseparasi mejadi sampel latih dan uji
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#### **Analisis Data**

Menampilkan tipe data dari sampel latih dan uji

In [ ]:
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

Menampilkan dimensi dari sampel latih dan uji

In [ ]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

Menelusuri salah satu citra

In [ ]:
index = 0
X_train[index]  # citra urutan pertama (indek ke-0) pada sampel latih

In [ ]:
# Menampilkan citra secara visual
img = plt.imshow(X_train[index])

In [ ]:
# Mengetahui label (numerik) dari citra
print('The image label is: ', y_train[index])

In [4]:
# Buat representasi label
label_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                'dog', 'frog', 'horse', 'ship', 'truck']

print('Number ', 'Label')
print('-------------------------')
for i in range(len(label_classes)):
    print(str(i).ljust(7), str(label_classes[i]).ljust(5))

Number  Label
-------------------------
0       airplane
1       automobile
2       bird 
3       cat  
4       deer 
5       dog  
6       frog 
7       horse
8       ship 
9       truck


In [ ]:
# Tampilkan label dari citra sebelumnya
print('The image class is: ', label_classes[y_train[index][0]])

#### **Praprosess**

***Label encoding* (dengan *one-hot encoding*)**

In [ ]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
# Hasil encoding pada label sampel latih
print(y_train_one_hot)

In [ ]:
# Hasil encoding pada label sampel uji
print(y_test_one_hot)

In [ ]:
# Menampilkan hasil encoding dari salah satu citra pada sampel latih
print('The one-hot label is: ', y_train_one_hot[0])

**Normalisasi Citra**

In [ ]:
X_train = X_train / 255     # sample latih
X_test = X_test / 255       # sampel uji

#### **Pemodelan**

**Bangun arsitektur model *Convolutional nwural network***

In [ ]:
model = Sequential()
# Convolutional and pooling layers
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
# Neural Nets dropout layers
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dense(10, activation='softmax'))

**Kompilasi model**

In [ ]:
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

#### **Pelatihan Model**

In [ ]:
hist = model.fit(X_train, y_train_one_hot,
                batch_size=256, epochs=10, 
                validation_split=0.2)

In [ ]:
# Evaluasi hasil pelatihan model
model.evaluate(X_test, y_test_one_hot)[1]

In [ ]:
# Memvisualisasikan akurasi dari model
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# Memvisualisasikan kerugian (loss) dari model
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

#### **Pengujian Model**

**Unggah citra**

In [ ]:
# Asumsinya citra uji diunggah pada Google Colab
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print('\nName of the image file: ',file_name)

# Membaca citra (3, 14, 20)
new_image = plt.imread(file_name)

**Tampilkan citra unggahan**

In [ ]:
img = plt.imshow(new_image)

**Ubah ukuran citra**

In [ ]:
# Transformasi ukuran (shape) citra dilakukan untuk menyesuikan
# input layer pada model
from skimage.transform import resize
resized_image = resize(new_image, (32, 32, 3))

**Buat prediksi**

In [ ]:
predictions = model.predict(np.array([resized_image]))

In [ ]:
# Tampilkan hasil prediksi
predictions

**Hasil Prediksi**

In [ ]:
# Urutkan probabilitas hasil prediksi terhadap kelas
list_index = [0,1,2,3,4,5,6,7,8,9]

x = predictions

for i in range(10):
  for j in range(10):
    if x[0][list_index[i]] > x[0][list_index[j]]:
      temp = list_index[i]
      list_index[i] = list_index[j]
      list_index[j] = temp

print(list_index)

In [ ]:
# Tampilkan 5 kelas dengan probabilitas teratas dari hasil prediksi
i = 0
for i in range(5):
  print(label_classes[list_index[i]],
        ':', 
        round(predictions[0][list_index[i]] * 100, 2),
        '%')

***

In [ ]:
model.save('185314098_cnn_model.h1')